<a href="https://colab.research.google.com/github/so-dipe/f1-dashboard/blob/main/F1%20dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install jupyter-dash
!pip install dash_bootstrap_components

In [ ]:
from jupyter_dash import JupyterDash
from dash import Dash, html, dcc
import dash
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime

import pandas as pd

In [ ]:
# Create App
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, 'https://codepen.io/chriddyp/pen/bWLwgP.css'])

In [ ]:
#Get Dataset
url2 = 'https://raw.githubusercontent.com/so-dipe/Web-Scraping-Datasets/main/f1-archive/f1-hybrid-era-results.csv'
url1 = 'https://raw.githubusercontent.com/so-dipe/Web-Scraping-Datasets/main/f1-archive/f1-data-for-model%20(1).csv'
df = pd.read_csv(url2)
df.head()

,Unnamed: 0,Pos,No,Driver,Car,Laps,Time/Retired,PTS,Circuit,Location,year,Q1,Q2,Q3,Starting Grid,race_num,Time
0,0,1,6,Nico Rosberg ROS,Mercedes,57.0,1:32:58.710,25.0,"Melbourne Grand Prix Circuit, Australia",Australia,2014,1:32.564,1:42.264,1:44.595,3,1,NaN
1,1,EX,3,Daniel Ricciardo RIC,Red Bull Racing Renault,57.0,+24.525s,0.0,"Melbourne Grand Prix Circuit, Australia",Australia,2014,1:30.775,1:42.295,1:44.548,2,1,NaN
2,2,2,20,Kevin Magnussen MAG,McLaren Mercedes,57.0,+26.777s,18.0,"Melbourne Grand Prix Circuit, Australia",Australia,2014,1:30.949,1:43.247,1:45.745,4,1,NaN
3,3,3,22,Jenson Button BUT,McLaren Mercedes,57.0,+30.027s,15.0,"Melbourne Grand Prix Circuit, Australia",Australia,2014,1:31.396,1:44.437,ELIMINATED,10,1,NaN
4,4,4,14,Fernando Alonso ALO,Ferrari,57.0,+35.284s,12.0,"Melbourne Grand Prix Circuit, Australia",Australia,2014,1:31.388,1:42.805,1:45.819,5,1,NaN


In [ ]:
def get_locations(dataframe, year):
  msk = (df['year'] == year)
  df_new = df[msk]
  ls = df_new['Location'].unique()
  dic = {i+1: ls[i] for i in range(0, len(ls), 1)}
  return dic

In [ ]:
# Title
title = html.Div(
    children=[
     html.H1('Formula 1 Visualization')   
    ]
)

In [ ]:
# Select Year
year_ls = [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
year = html.Div(
    children=[
        html.Label('Year'),
        dcc.Dropdown(
            options=year_ls, value=2014, id='year'
        )
    ]
)

In [ ]:
#create slider for grand-prix
# def get_slider(year):
#   marks = get_locations(df, year)
#   slider = html.Div(
#       [
#           html.Label('Grand Prix'),
#           dcc.Slider(
#               step=1, marks=marks, value=1, id='grand-prix'
#           )
#       ]
#   )
#   return slider

slider = html.Div(
      [
          html.Label('Grand Prix'),
          dcc.Slider(
              step=1, value=1, id='grand-prix'
          )
      ]
  )

In [ ]:
#serve slider
@app.callback(
    Output('grand-prix', 'marks'),
    Input('year', 'value')
)

def serve_slider(year):
  return get_locations(df,year)

In [ ]:
#put year and gp slider in one row
row_1 = html.Div(
    children=[
        dbc.Row(
            [
            dbc.Col(year, width=1, align='left'),
            dbc.Col(slider, align='right')
            ]
        )
    ]
)

In [ ]:
#get drivers championship dataframe
def wdc_pts(year, race_num):
  year_msk = df['year'] == year
  race_msk = df['race_num'] <= race_num
  df_new = df[year_msk & race_msk]
  race_pts = pd.pivot_table(df_new, index='Driver', columns='race_num')['PTS']
  for col in race_pts.columns:
    if col == 1:
      pass
    else:
      race_pts[col] = race_pts[[col-1, col]].sum(axis=1)
  race_pts.columns = df_new[df_new['race_num'] <= race_num]['Location'].unique()
  return race_pts 

In [ ]:
#get constructors championship dataframe
def wcc_pts(year, race_num):
  year_msk = df['year'] == year
  race_msk = df['race_num'] <= race_num
  df_new = df[year_msk & race_msk]
  race_pts = pd.pivot_table(df_new, index='Car', columns='race_num', aggfunc='sum')['PTS']
  for col in race_pts.columns:
    if col == 1:
      pass
    else:
      race_pts[col] = race_pts[[col-1, col]].sum(axis=1)
  race_pts.columns = df_new[df_new['race_num'] <= race_num]['Location'].unique()
  return race_pts

In [ ]:
def plot_driver_pts(year, race_num, drivers):
  fig = go.Figure()
  race_pts = wdc_pts(year, race_num)
  for driver in drivers:
    driver_short = driver[-3:]
    driver_pts = race_pts.loc[driver, :]
    fig.add_trace(
        go.Scatter(x=driver_pts.index, y=driver_pts.values, line_shape='linear', name=driver_short)
    )
  annotations = []
  annotations.append(dict(x=0.5, y=-0.1,
                              xanchor='center', yanchor='top',
                              text='Drivers that (on rare occasions) start from the pitlane are not included in the points',
                              font=dict(family='Arial',
                                        size=12,
                                        color='rgb(150,150,150)'),
                              showarrow=False))
  fig.update_layout(
      height=550,
      width=1500,
      margin={'t':25, 'r':0, 'l':30, 'b':0},
      hoverlabel=dict(
          font_size=16          
      ),
      title_text='World Drivers Championship',
      annotations=annotations
  )
  return fig

In [ ]:
# def get_wdc_fig(year, race_num, drivers):
#   fig = go.Figure()
#   # driver = ['Lewis  Hamilton  HAM', 'Nico  Rosberg  ROS'] 
#   figure = plot_driver_pts(year, race_num, drivers)
#   wdc_fig = html.Div([
#     dcc.Graph(id='wdc-fig', figure=figure)
#   ])
#   return wdc_fig

wdc_fig = html.Div([
    dcc.Graph(id='wdc-fig')
  ])

In [ ]:
@app.callback(
    Output('wdc-fig', 'figure'),
    Input('year', 'value'),
    Input('grand-prix', 'value'),
    Input('driver-check', 'value')
)
def serve_wdc_fig(year, race_num, drivers):
  race_num = int(race_num)
  figure = plot_driver_pts(year, race_num, drivers)
  return figure#get_wdc_fig(year, race_num, drivers)

In [ ]:
# def get_drivers(year, race_num):
#   driver_ls = list()
#   drivers = wdc_pts(year, race_num).index
#   for driver in drivers:
#     driver_ls.append(driver) 
#   check = html.Div([
#       dcc.Checklist(driver_ls, driver_ls,id='driver-check', labelStyle={'display': 'block'})
#   ])
#   return check

check = html.Div([
      dcc.Checklist(id='driver-check', labelStyle={'display': 'block'})
  ])

In [ ]:
@app.callback(
    Output('driver-check', 'options'),
    Input('year', 'value'),
    Input('grand-prix', 'value')
)
def serve_driver_checklist(year, race_num):
  driver_ls = list()
  drivers = wdc_pts(year, race_num).index
  for driver in drivers:
    driver_ls.append(driver)
  return driver_ls #get_drivers(year, race_num)

In [ ]:
@app.callback(
    Output('driver-check', 'value'),
    Input('year', 'value'),
    Input('grand-prix', 'value')
)
def serve_driver_checked(year, race_num):
  driver_ls = list()
  drivers = wdc_pts(year, race_num).index
  for driver in drivers:
    driver_ls.append(driver)
  return driver_ls #get_drivers(year, race_num)

In [ ]:
# Create Drivers Checklist
# wdc_check = html.Div(id='wdc-check')

In [ ]:
# Create Second row
row_2 = html.Div(
    dbc.Row([
        dbc.Col(wdc_fig, align='left'),
        dbc.Col(check, id='check', width=2, align='right')
    ])
)

In [ ]:
def plot_pos_change(year, race_num):
  df_ = df.copy()
  df_['Pos'] = df_['Pos'].apply(str)
  year_msk = df_['year'] == year
  race_msk = df_['race_num'] == race_num
  pos_df = df_[year_msk & race_msk]
  pos_df = pos_df[['Driver', 'Pos', 'Starting Grid']]
  pos_df['Starting Grid'] = pos_df['Starting Grid'].apply(int)
  num_ls = '0123456789'
  for num in pos_df['Pos'].unique():
    # print(type(num))
    if num[0] not in num_ls:
      index = pos_df[pos_df['Pos'] == num].index
      pos_df.loc[index, 'Pos'] = len(pos_df) + 1
  pos_df['Pos'] = pos_df['Pos'].apply(int)
  fig = go.Figure()
  for i in pos_df.index:
    driver = pos_df.loc[i]
    if driver['Pos'] == driver['Starting Grid']:
      fig.add_trace(
          go.Scatter(
          x=[driver['Driver'], driver['Driver']],
          y=[driver['Pos'], driver['Starting Grid']],
          marker=dict(color='rgb(0, 0, 255)', size=10),
          name=None
        )
      )
    elif int(driver['Pos']) < driver['Starting Grid']:
      fig.add_trace(
        go.Scatter(
            x=[driver['Driver'], driver['Driver']],
            y=[driver['Pos'], driver['Starting Grid']],
            marker=dict(color='rgb(0, 255, 0)'),
            name=None
        )
      )
    elif int(driver['Pos']) > driver['Starting Grid']:
        fig.add_trace(
          go.Scatter(
            x=[driver['Driver'], driver['Driver']],
            y=[driver['Pos'], driver['Starting Grid']],
            marker=dict(color='rgb(255, 0, 0)', size=10),
            name=None
        )
      )
  fig.add_hline(len(pos_df) + 1)
  annotations=[]
  annotations.append(dict(x=0.5, y=-0.1,
                              xanchor='center', yanchor='top',
                              text='Red represents a drop in position' + 
                          ' and Green a positive change, the black line represents' +
                          ' Drivers that retired or got disqualified (NC/DNF/EX)',
                              font=dict(family='Arial',
                                        size=12,
                                        color='rgb(150,150,150)'),
                              showarrow=False))
  fig.update_layout(
      xaxis=dict(
          showgrid=False,
          linewidth=5,
          ticks='outside'
      ),
      showlegend=False,
      width=700,
      height=600,
      title_text='Drivers Position Change (Start to Finish)',
      margin={'r':0, 'b':0, 't':25, 'l':30}
      # annotations=annotations
  )
  return fig

In [ ]:
# def get_pos_change(year, race_num):
#   figure = plot_pos_change(year, race_num)
#   plot_html = dcc.Graph(figure=figure)
#   return plot_html

pos_change = html.Div([dcc.Graph(id='pos-change')])

In [ ]:
# pos_change = html.Div(id='pos-change')

In [ ]:
@app.callback(
    Output('pos-change', 'figure'),
    Input('year', 'value'),
    Input('grand-prix', 'value')
)
def serve_pos_change(year=2014, race_num=1):
  figure = plot_pos_change(year, race_num)
  return figure#get_pos_change(year, race_num)

In [ ]:
def str_to_time(time_str):
  # num_ls = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
  if time_str[0] not in '0123456789':
    time_str = '0:00.0'
    time_str = datetime.strptime(time_str, '%M:%S.%f').time()
  elif ':' not in time_str:
    time_str = datetime.strptime(time_str, '%S.%f').time()
  else:
    time_str = datetime.strptime(time_str, '%M:%S.%f').time()
  return time_str

def time_to_int(time):
  time = (
      time.microsecond +
      (time.second * 1_000_000) + 
      (time.minute * 60 * 1_000_000)
  )
  return time

df['Q1 time'] = df['Q1'].apply(str_to_time).apply(time_to_int)
df['Q2 time'] = df['Q2'].apply(str_to_time).apply(time_to_int)
df['Q3 time'] = df['Q3'].apply(str_to_time).apply(time_to_int)

In [ ]:
def plot_quali_session(year, race_num, session):
  fig = go.Figure()
  year_msk = df['year'] == year
  race_msk = df['race_num'] == race_num
  session_t = session + ' time'
  df_new = df[year_msk & race_msk][['Driver', session,session_t]]
  zero_msk = df_new[session_t] == 0
  df_zero = df_new[zero_msk]
  df_new = df_new[~zero_msk]
  df_new[session_t] = (
      (df_new[session_t] - df_new[session_t].min())
      / 1_000_000
      # /(df_new[session_t].max() - df_new[session_t].min())
  )
  df_new.sort_values(by=session_t, ascending=False, inplace=True)
  if '1' in session:
    df_elim = df_new.head(5 - len(df_zero))
    df_next = df_new.drop(index=df_elim.index)
    df_elim = pd.concat([df_zero, df_elim])
    fig.add_trace(
        go.Bar(x=df_elim[session_t], y=df_elim['Driver'], orientation='h', marker=dict(color='rgb(255, 0, 0)'), name='Eliminated')
    )
    fig.add_trace(
        go.Bar(x=df_next[session_t], y=df_next['Driver'], orientation='h', marker=dict(color='rgb(0, 255, 0)'))
    )
  elif '2' in session:
    elim_msk = df_zero[session] == 'ELIMINATED'
    df_zero = df_zero[~elim_msk]
    df_elim = df_new.head(5 - len(df_zero))
    df_next = df_new.drop(index=df_elim.index)
    df_elim = pd.concat([df_zero, df_elim])
    fig.add_trace(
        go.Bar(x=df_elim[session_t], y=df_elim['Driver'], orientation='h', marker=dict(color='rgb(255, 0, 0)'), name='Eliminated')
    )
    fig.add_trace(
        go.Bar(x=df_next[session_t], y=df_next['Driver'], orientation='h', marker=dict(color='rgb(0, 255, 0)'))
    )
  elif '3' in session:
    elim_msk = df_zero[session] == 'ELIMINATED'
    df_zero = df_zero[~elim_msk]
    fig.add_trace(
        go.Bar(x=df_new[session_t], y=df_new['Driver'], orientation='h', marker=dict(color='rgb(0, 255, 0)'))
    )
  fig.update_layout(
      showlegend=False,
      margin={'r':5, 'b':0},
      title_text=f'Qualifying {session[1]} Results',
      xaxis_title='delta time (seconds)'
  )
  return fig

In [ ]:
def get_q_plot(year, race_num, session):
  figure = plot_quali_session(year, race_num, session)
  q1_plot = html.Div([
    dcc.Graph(id='q1-plot', figure=figure)
  ])
  return q1_plot

In [ ]:
@app.callback(
    Output('plot-q1', 'children'),
    Input('year', 'value'),
    Input('grand-prix', 'value')
)
def serve_q1_plot(year, race_num):
  return get_q_plot(year, race_num, 'Q1')
plot_q1 = html.Div(id='plot-q1')

In [ ]:
@app.callback(
    Output('plot-q2', 'children'),
    Input('year', 'value'),
    Input('grand-prix', 'value')
)
def serve_q2_plot(year, race_num):
  return get_q_plot(year, race_num, 'Q2')
plot_q2 = html.Div(id='plot-q2')

In [ ]:
@app.callback(
    Output('plot-q3', 'children'),
    Input('year', 'value'),
    Input('grand-prix', 'value')
)
def serve_q3_plot(year, race_num):
  return get_q_plot(year, race_num, 'Q3')
plot_q3 = html.Div(id='plot-q3')

In [ ]:
row4 = html.Div([
    dbc.Row([
        dbc.Col([plot_q1]),
        dbc.Col([plot_q2]),
        dbc.Col([plot_q3])
    ])
])

In [ ]:
pos_dict = {
    1:'1st', 2:'2nd', 3:'3rd', 4:'4th', 5:'5th',
    6:'6th', 7:'7th', 8:'8th', 9:'9th', 10:'10th',
    11:'11th', 12:'12th', 13:'13th', 14:'14th', 15:'15th',
    17:'17th', 18:'18th', 19:'19th', 20:'20th', 21:'21th', 22:'22th', 30:'DNF/NC/EX/DQ'
}
def driver_h2h(year, race_num, car):
  df_ = df.copy()
  df_['Pos'] = df_['Pos'].replace({'NC':30, 'DQ':30, 'EX':30}).apply(int)
  year_msk = df_['year'] == year
  race_msk = df_['race_num'] <= race_num
  car_msk = df_['Car'] == car
  df_new = df_[year_msk & race_msk & car_msk]
  df_new = pd.pivot_table(df_new, index='Driver', columns='race_num').drop(columns=['Laps', 'year', 'No', 'Unnamed: 0'])
  dnf = (df_new['Pos'] == 30).sum(axis=1)
  # df_new['PTS'] = df_new['PTS']/df_new['PTS'].max()
  pts = df_new['PTS'].sum(axis=1)
  # pts = (pts / pts.max()) * 15
  best_start = df_new['Starting Grid'].min(axis=1)
  best_finish = df_new['Pos'].min(axis=1)
  driver1_quali = (df_new['Starting Grid'].iloc[0] < df_new['Starting Grid'].iloc[1]).sum()
  driver2_quali = (df_new['Starting Grid'].iloc[0] > df_new['Starting Grid'].iloc[1]).sum()
  quali_comp = pd.Series([driver1_quali, driver2_quali], index=df_new.index)
  driver1_race = (df_new['Pos'].iloc[0] < df_new['Pos'].iloc[1]).sum()
  driver2_race = (df_new['Pos'].iloc[0] > df_new['Pos'].iloc[1]).sum()
  race_comp = pd.Series([driver1_race, driver2_race], index=df_new.index)
  df_new = pd.DataFrame(pts, index=df_new.index).rename(columns={0:'Points'})
  df_new['Best Race Start'] = best_start
  df_new['Qualifying'] = quali_comp
  df_new['Best Finish'] = best_finish
  df_new['DNFs'] = dnf
  df_new['Race'] = race_comp
  return df_new.fillna(0)

def plot_h2h(year, race_num, car):
  plot_df = driver_h2h(year, race_num, car)
  plot_df['scale pts'] = (plot_df['Points'] / plot_df['Points'].max()) * 15
  plot_df.iloc[1] = -plot_df.iloc[1]
  colors = ['rgb(212, 12, 128)', 'rgb(12, 112, 128)']
  # plot_df['Best Finish'].replace(1:'1st', 2:'2nd', 3:'3rd')
  x_driver1 = [
      plot_df.iloc[0]['Race'],
      plot_df.iloc[0]['scale pts'],
      plot_df.iloc[0]['Best Race Start'],
      plot_df.iloc[0]['Best Finish'],
      plot_df.iloc[0]['Qualifying'],
      plot_df.iloc[0]['DNFs'],
  ]
  x_driver1.reverse()
  text_driver1 = [
      round(plot_df.iloc[0]['Race']),
      plot_df.iloc[0]['Points'],
      round(plot_df.iloc[0]['Best Race Start']),
      plot_df['Best Finish'].apply(round).replace(pos_dict).iloc[0],
      round(plot_df.iloc[0]['Qualifying']),
      round(plot_df.iloc[0]['DNFs'])
  ]
  text_driver1.reverse()
  x_driver2 = [
      plot_df.iloc[1]['Race'],
      plot_df.iloc[1]['scale pts'],
      plot_df.iloc[1]['Best Race Start'],
      plot_df.iloc[1]['Best Finish'],
      plot_df.iloc[1]['Qualifying'],
      plot_df.iloc[1]['DNFs']
  ]
  x_driver2.reverse()
  text_driver2 = [
      round(-plot_df.iloc[1]['Race']),
      -plot_df.iloc[1]['Points'],
      round(-plot_df.iloc[1]['Best Race Start']),
      (-plot_df['Best Finish']).apply(round).replace(pos_dict).iloc[1],
      round(-plot_df.iloc[1]['Qualifying']),
      round(-plot_df.iloc[1]['DNFs'])
  ]
  text_driver2.reverse()
  y=['Race', 'Points', 'Best Race Start', 'Best Finish', 'Qualifying', 'DNFs']
  y.reverse()
  # y=['DNFs', 'Qualifying', 'Best Finish', 'Best Race Start', 'Points', 'Race']
  fig = go.Figure()
  fig.add_trace(
      go.Bar(x=x_driver1, y=y, orientation='h', name=plot_df.index[0], text=text_driver1)
  )
  fig.add_trace(
      go.Bar(x=x_driver2, y=y, orientation='h', name=plot_df.index[1], text=text_driver2)
  )
  fig.update_traces(textposition='outside')
  annotations = []
  annotations.append(dict(x=0.5, y=-0.1,
                              xanchor='center', yanchor='top',
                              text='Best Finish 30 actually means a DNF',
                              font=dict(family='Arial',
                                        size=12,
                                        color='rgb(150,150,150)'),
                              showarrow=False))
  fig.update_layout(
      margin={'r':0, 'b':0, 't':25, 'l':0},
      title_text=f'{car} drivers Head to Head results',
      width=900,
      xaxis=dict(visible=False)
      # annotations=annotations,
  )
  return fig

In [ ]:
# def get_h2h(year, race_num):
#   cars = df[df['year'] == year]['Car'].unique()
#   row_num = round(len(cars)/2)
#   row1_cars = cars[:row_num]
#   row2_cars = cars[row_num:]
#   row1_children = list()
#   for car in row1_cars:
#     figure = plot_h2h(year, race_num, car)
#     car_graph = dbc.Row([dcc.Graph(figure=figure)])
#     row1_children.append(car_graph)
#   row2_children = list()
#   for car in row2_cars:
#     figure = plot_h2h(year, race_num, car)
#     car_graph = dbc.Row([dcc.Graph(figure=figure)])
#     row2_children.append(car_graph)
#   grouped = html.Div([
#       dbc.Row([
#           dbc.Col(row1_children),
#           dbc.Col(row2_children)
#       ])
#   ])
#   return grouped

cars_radio = html.Div([
    dcc.RadioItems(id='cars', value='Mercedes')
])

def get_cars(year):
  return df[df['year'] == year]['Car'].unique()

In [ ]:
@app.callback(
    Output('cars', 'options'),
    Input('year', 'value')
)
def serve_cars(year):
  return get_cars(year)

In [ ]:
show_h2h = html.Div([
    dcc.Graph(id='h2h')
])

In [ ]:
@app.callback(
    Output('h2h', 'figure'),
    Input('year', 'value'),
    Input('grand-prix', 'value'),
    Input('cars', 'value')
)
def serve_h2h(year, race_num, car):
  figure = plot_h2h(year, race_num, car)
  return figure

In [ ]:
row_7 = html.Div([
    dbc.Row([
        dbc.Col(pos_change, align='left'),
        dbc.Col(show_h2h, align='center'),
        dbc.Col(cars_radio, width=1, align='right')
    ])
])

In [ ]:
break_line = html.Br()
# Add components
app.layout = html.Div(
    children=[
        title,
        break_line,
        row_1,
        break_line,
        row_2,
        break_line,
        row_7,
        break_line,
        row4, 
        break_line,
        # row_7,
        # pos_h2h
    ]      
)

In [ ]:
app.run_server(mode='external')

Dash app running on:


<IPython.core.display.Javascript object>

In [ ]:
# Run App
# app.run_server(mode='inline', debug=True)

In [ ]:
! pip freeze

absl-py==1.2.0
aiohttp==3.8.1
aiosignal==1.2.0
alabaster==0.7.12
albumentations==1.2.1
altair==4.2.0
ansi2html==1.8.0
appdirs==1.4.4
arviz==0.12.1
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
async-timeout==4.0.2
asynctest==0.13.0
atari-py==0.2.9
atomicwrites==1.4.1
attrs==22.1.0
audioread==2.1.9
autograd==1.4
Babel==2.10.3
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==5.0.1
blis==0.7.8
bokeh==2.3.3
branca==0.5.0
Brotli==1.0.9
bs4==0.0.1
CacheControl==0.12.11
cached-property==1.5.2
cachetools==4.2.4
catalogue==2.0.8
certifi==2022.6.15
cffi==1.15.1
cftime==1.6.1
chardet==3.0.4
charset-normalizer==2.1.0
click==7.1.2
clikit==0.6.2
cloudpickle==1.3.0
cmake==3.22.6
cmdstanpy==1.0.4
colorcet==3.0.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.4.0
crashtest==0.3.1
crcmod==1.7
cufflinks==0.17.3
cvxopt==1.3.0
cvxpy==1.2.1
cycler==0.11.0
cymem==2.0.6
Cython==0.29.32
daft==0.0.4
dash==2.6.1
dash-bootstrap-components==1.2.1
dash-core-components==2.0.0
dash-html-component